In [3]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics.pairwise import cosine_distances



In [4]:
#!{sys.executable} -m pip install pandas numpy scikit-learn

In [ ]:
def random_data_gen(num_rows = 1000, n_layer_1 = 3, n_layer_2 = 20, start_dt = datetime(2020, 1, 1), end_dt = datetime(2025, 1, 1) ):
    id1 = ["L1_"+str(x) for x in np.random.randint(n_layer_1, size=num_rows)]
    id2 = ["L2_"+str(x) for x in np.random.randint(n_layer_2, size=num_rows)]
    start_u = start_dt.timestamp()
    end_u = end_dt.timestamp()
    random_ts = np.random.uniform(start_u, end_u, num_rows)
    dt = sorted([datetime.fromtimestamp(ts) for ts in random_ts])

    df = pd.DataFrame({
        'id1': id1,
        'id2': id2,
        'timestamp': dt
    })
    return df   

In [ ]:
df = random_data_gen()
df['timeframe'] = df['timestamp'].dt.to_period('Y').apply(lambda r: str(r.start_time)+" to " + str(r.end_time))
df

,id1,id2,timestamp,timeframe
0,L1_0,L2_16,2020-01-01 12:54:49.363446,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...
1,L1_0,L2_18,2020-01-03 01:51:05.991340,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...
2,L1_0,L2_2,2020-01-04 20:36:48.651503,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...
3,L1_0,L2_7,2020-01-04 22:59:45.491460,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...
4,L1_1,L2_0,2020-01-08 04:22:57.980675,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...
...,...,...,...,...
995,L1_0,L2_0,2024-12-26 11:43:55.312047,2024-01-01 00:00:00 to 2024-12-31 23:59:59.999...
996,L1_1,L2_13,2024-12-27 21:50:55.017456,2024-01-01 00:00:00 to 2024-12-31 23:59:59.999...
997,L1_0,L2_14,2024-12-28 18:22:15.887553,2024-01-01 00:00:00 to 2024-12-31 23:59:59.999...
998,L1_0,L2_10,2024-12-29 18:42:32.891525,2024-01-01 00:00:00 to 2024-12-31 23:59:59.999...


In [ ]:
grouped_df = df.groupby(['id1','id2','timeframe']).size().reset_index(name='counts')
grouped_df

,id1,id2,timeframe,counts
0,L1_0,L2_0,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...,2
1,L1_0,L2_0,2021-01-01 00:00:00 to 2021-12-31 23:59:59.999...,4
2,L1_0,L2_0,2022-01-01 00:00:00 to 2022-12-31 23:59:59.999...,4
3,L1_0,L2_0,2023-01-01 00:00:00 to 2023-12-31 23:59:59.999...,4
4,L1_0,L2_0,2024-01-01 00:00:00 to 2024-12-31 23:59:59.999...,4
...,...,...,...,...
289,L1_2,L2_9,2020-01-01 00:00:00 to 2020-12-31 23:59:59.999...,4
290,L1_2,L2_9,2021-01-01 00:00:00 to 2021-12-31 23:59:59.999...,4
291,L1_2,L2_9,2022-01-01 00:00:00 to 2022-12-31 23:59:59.999...,2
292,L1_2,L2_9,2023-01-01 00:00:00 to 2023-12-31 23:59:59.999...,3


In [ ]:
# Pivot grouped_df so that id2 becomes columns
pivot_df = grouped_df.pivot(index=['id1', 'timeframe'], columns='id2', values='counts').fillna(0)
pivot_df

id2                                                      L2_0  L2_1  L2_10  \
id1  timeframe                                                               
L1_0 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...   2.0   5.0    3.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...   4.0   4.0    5.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...   4.0   5.0    2.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...   4.0   3.0    4.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...   4.0   1.0    2.0   
L1_1 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...   2.0   1.0    3.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...   3.0   2.0    4.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...   5.0   4.0    3.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...   4.0   4.0    7.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...   1.0   1.0    4.0   
L1_2 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...   6.0   3.0    2.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...   1.0   2.0    4.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...   6.0   3.0    3.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...   3.0   3.0    4.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...   2.0   4.0    2.0   

id2                                                      L2_11  L2_12  L2_13  \
id1  timeframe                                                                 
L1_0 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...    1.0    1.0    2.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...    5.0    6.0    4.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...    7.0    3.0    2.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...    5.0    3.0    5.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...    3.0    5.0    1.0   
L1_1 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...    3.0    3.0    3.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...    4.0    3.0    3.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...    4.0    1.0    2.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...    1.0    3.0    2.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...    4.0    2.0    7.0   
L1_2 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...    1.0    8.0    1.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...    5.0    4.0    2.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...    1.0    4.0    3.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...    5.0    4.0    4.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...    3.0    1.0    2.0   

id2                                                      L2_14  L2_15  L2_16  \
id1  timeframe                                                                 
L1_0 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...    2.0    4.0    4.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...    1.0    1.0    2.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...    4.0    3.0    4.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...    5.0    3.0    3.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...    3.0    3.0    4.0   
L1_1 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...    2.0    3.0    8.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...    3.0    2.0    3.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...    4.0    8.0    1.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...    2.0    2.0    2.0   
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...    4.0    4.0    3.0   
L1_2 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...    5.0    3.0    8.0   
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...    6.0    2.0    2.0   
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...    4.0    3.0    4.0   
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...    2.0    6.0    4.0   
     2024-01-01 00:00:00 to 2024

In [ ]:
cosine_dist_matrix = pd.DataFrame(cosine_distances(pivot_df), index=pivot_df.index, columns=pivot_df.index)
cosine_dist_matrix

id1                                                                                                     L1_0  \
timeframe                                               2020-01-01 00:00:00 to 2020-12-31 23:59:59.999999999   
id1  timeframe                                                                                                 
L1_0 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...                                           0.000000     
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...                                           0.225863     
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...                                           0.302531     
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...                                           0.300386     
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           0.258724     
L1_1 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...                                           0.230325     
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...                                           0.228635     
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...                                           0.229447     
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...                                           0.221622     
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           0.220249     
L1_2 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...                                           0.209613     
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...                                           0.254731     
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...                                           0.228551     
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...                                           0.224441     
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           0.274953     

id1                                                                                                           \
timeframe                                               2021-01-01 00:00:00 to 2021-12-31 23:59:59.999999999   
id1  timeframe                                                                                                 
L1_0 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...                                           0.225863     
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...                                           0.000000     
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...                                           0.219983     
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...                                           0.162422     
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           0.199578     
L1_1 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...                                           0.168093     
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...                                           0.105391     
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...                                           0.208436     
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...                                           0.131079     
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...                                           0.213389     
L1_2 2020-01-01 00:00:00 to 2020-12-31 23:59:59.9999...                                           0.217544     
     2021-01-01 00:00:00 to 2021-12-31 23:59:59.9999...                                           0.138031     
     2022-01-01 00:00:00 to 2022-12-31 23:59:59.9999...                                           0.174576     
     2023-01-01 00:00:00 to 2023-12-31 23:59:59.9999...                                           0.165638     
     2024-01-01 00:00:00 to 2024-12-31 23:59:59.9999...                        